In [1]:
import tensorflow as tf
import helper as helper

_helper = helper.helper()

In [2]:
from keras import Sequential, Model
from keras.utils import plot_model as pltmdl
from keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Input
from keras.layers import GlobalAveragePooling2D, RandomFlip, RandomCrop
from keras.layers import RandomRotation, RandomZoom, RandomHeight, RandomWidth
from keras.layers import Rescaling
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator as IDG
from keras.utils import image_dataset_from_directory

In [6]:
from importlib import reload
reload(sys.modules["helper"])

<module 'helper' from 'd:\\Documents\\GitHub\\Sparta-system_maintenance\\Machine_Learning\\helper.py'>

In [7]:
try:
  _helper.data_preparations.zip_processing('https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip',
                                            '10_food_classes_10_percent_a')
except Exception as e:
  print(f"Error: {e}")
  _helper.notifications.error()

Downloaded, Extracted and Saved. Directory: ./resources/10_food_classes_10_percent_a.zip


In [3]:
try:
  test_data, train_data, CLASS_NAMES = _helper.data_preparations.train_test_dir_setter("./resources/Actual/",IMG_SIZE = (224, 224),CLASS_MODE = "categorical")
except Exception as e:
  print(f"Error: {e}")
  _helper.notifications.error()

Test Data: 
Found 30 files belonging to 3 classes.
Train Data: 
Found 156 files belonging to 3 classes.
Class Names: ['exportable', 'marketable', 'reject']


In [38]:
try:
  checkpoint, ckpt_dir = _helper.callbacks.checkpoint_callback(
    "./saved_files/EffNetV2L",
    "Model_1",
    "val_categorical_accuracy",
    True,
    True
  )
except Exception as e:
  print(f"Error: {e}")
  _helper.notifications.error()

In [39]:
try:
  model_parts = tf.keras.applications.EfficientNetV2L(include_top = False)

  model_parts.trainable = False

  input = Input(shape = (224, 224, 3), name = "input_layer")

  x = model_parts(input)
  x = GlobalAveragePooling2D(name = "GAP2D_layer")(x)
  output = Dense(3, activation = "softmax", name = "output_layer")(x)

  model_test = Model(input, output, name = "EffNetV2L")

  model_test.compile(
    loss = "categorical_crossentropy",
    optimizer = Adam(),
    metrics = ["CategoricalAccuracy", "Precision"]
  )

  model_test_h = model_test.fit(train_data,
                          epochs = 15,
                          steps_per_epoch = len(train_data),
                          validation_data = test_data,
                          callbacks = [checkpoint,
                                      _helper.callbacks.tensorboard_callback("./log/EffNetV2L",
                                                                              "Model 1")])
  _helper.notifications.model_callback_notification()
except Exception as e:
  print(f"Error: {e}")
  _helper.notifications.error()

473176280/473176280 [==============================] - 130s 0us/step
Saving TensorBoard Log Files to: ./log/EffNetV2L/Model 1/2022_10_29__01_25_log
Epoch 1/15
5/5 [==============================] - 118s 17s/step - loss: 1.1258 - categorical_accuracy: 0.2436 - precision: 0.0000e+00 - val_loss: 1.1165 - val_categorical_accuracy: 0.3000 - val_precision: 0.0000e+00
Epoch 2/15
5/5 [==============================] - 69s 15s/step - loss: 1.0784 - categorical_accuracy: 0.4359 - precision: 0.0000e+00 - val_loss: 1.0998 - val_categorical_accuracy: 0.4000 - val_precision: 0.0000e+00
Epoch 3/15
5/5 [==============================] - 72s 15s/step - loss: 1.0228 - categorical_accuracy: 0.5385 - precision: 1.0000 - val_loss: 1.0825 - val_categorical_accuracy: 0.5000 - val_precision: 0.0000e+00
Epoch 4/15
5/5 [==============================] - 75s 16s/step - loss: 0.9908 - categorical_accuracy: 0.6538 - precision: 0.0000e+00 - val_loss: 1.0688 - val_categorical_accuracy: 0.6000 - val_precision: 0.0000

In [40]:
model_test.evaluate(test_data)

1/1 [==============================] - 9s 9s/step - loss: 0.9650 - categorical_accuracy: 0.6000 - precision: 0.8000


[0.964994490146637, 0.6000000238418579, 0.800000011920929]

In [41]:
try:
  _helper.save_load.save(model = model_test,
                        path = "./saved_files/EffNetV2L",
                        name = "Model 1")
except Exception as e:
  print(f"Error {e}")
  _helper.notifications.error()

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.
Model is saved to ./saved_files/EffNetV2L/Model 1_2022_10_29 - 01.h5 and ./saved_files/EffNetV2L/Model 1_diagram.png
